In [14]:
import pandas as pd
import numpy as np
import fasttext
from tqdm.auto import tqdm

In [9]:
path_to_pretrained_model = '../lid.176.bin'
fmodel = fasttext.load_model(path_to_pretrained_model)
lang, prob = fmodel.predict('testua bla bla eu')  # ([['__label__en']], [array([0.9331119], dtype=float32)]

In [8]:
%%time
data = pd.read_csv('../tweets.csv', sep=';', parse_dates=['timestamp'])
# remove blank tweets
data = data[~data['text'].isna()]
print(len(data))
data.head()

/home/ubuntu/anaconda3/lib/python3.8/site-packages/decorator.py:231: DtypeWarning: Columns (0,1,2,3,8) have mixed types.Specify dtype option on import or set low_memory=False.
  return caller(func, *(extras + args), **kw)


18809800
CPU times: user 2min 28s, sys: 9.31 s, total: 2min 37s
Wall time: 2min 37s


,id,user,fullname,url,timestamp,replies,likes,retweets,text
0,1132977055300300800,KamdemAbdiel,Abdiel kamdem,NaN,2019-05-27 11:49:14+00:00,0.0,0.0,0.0,È appena uscito un nuovo video! LES CRYPTOMONN...
1,1132977073402736640,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00:00,0.0,0.0,0.0,Cardano: Digitize Currencies; EOS https://t.co...
2,1132977023893139456,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00:00,0.0,2.0,1.0,Another Test tweet that wasn't caught in the s...
3,1132977089089556481,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00:00,0.0,0.0,0.0,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...
4,1132977092340191232,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00:00,0.0,0.0,0.0,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...


In [31]:
# only tweets with +20 likes
df = data[data['likes'] > 20]
df['sentiment'] = ''
print(len(df))

284466


<ipython-input-31-a800604a4e1c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = ''


In [32]:
df['language'] = ''
df['probability'] = 0

for index, row in tqdm(df.iterrows(), total=len(df)):
    lang, prob = fmodel.predict(row['text'].replace('\n',''))
    df.loc[index, 'language'] = lang[0]
    df.loc[index, 'probability'] = prob[0]

<ipython-input-32-c864af86ab2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['language'] = ''
<ipython-input-32-c864af86ab2e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['probability'] = 0


  0%|          | 0/284466 [00:00<?, ?it/s]

/home/ubuntu/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [34]:
df['language'].value_counts()

__label__en    230314
__label__tr     28322
__label__ja     15411
__label__es      3478
__label__pt      1546
                ...  
__label__kn         1
__label__gd         1
__label__bn         1
__label__nn         1
__label__ht         1
Name: language, Length: 68, dtype: int64

In [36]:
df[df['language'] == '__label__en'].to_csv('tweets_20likes_eng.csv', index=False)

In [37]:
df2 = df[df['language'] == '__label__en']

In [38]:
import torch
import torch.nn.functional as F
from pytorch_pretrained_bert import BertTokenizer
from bertModel import BertClassification
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [39]:
labels = {0:'neutral', 1:'positive',2:'negative'}
num_labels= len(labels)
vocab = "finance-uncased"
vocab_path = 'analyst_tone/vocab'
pretrained_weights_path = "analyst_tone/pretrained_weights" # this is pre-trained FinBERT weights
fine_tuned_weight_path = "analyst_tone/fine_tuned.pth"      # this is fine-tuned FinBERT weights
max_seq_length=512
device='cuda'
tokenizer = BertTokenizer(vocab_file = vocab_path, do_lower_case = True, do_basic_tokenize = True)

In [40]:
model = BertClassification(weight_path= pretrained_weights_path, num_labels=num_labels, vocab=vocab)
model.load_state_dict(torch.load(fine_tuned_weight_path, map_location=torch.device(device)))
model.to(device);
model.eval();

/home/ubuntu/dl_final_project/FinBERT/bertModel.py:31: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(self.classifier.weight)


In [41]:
for index, row in tqdm(df2.iterrows(), total=len(df2)):
    sent = row['text']
    tokenized_sent = tokenizer.tokenize(sent)
    if len(tokenized_sent) > max_seq_length:
        tokenized_sent = tokenized_sent[:max_seq_length]
    
    ids_review  = tokenizer.convert_tokens_to_ids(tokenized_sent)
    mask_input = [1]*len(ids_review)        
    padding = [0] * (max_seq_length - len(ids_review))
    ids_review += padding
    mask_input += padding
    input_type = [0]*max_seq_length
    
    input_ids = torch.tensor(ids_review).to(device).reshape(-1, max_seq_length)
    attention_mask =  torch.tensor(mask_input).to(device).reshape(-1, max_seq_length)
    token_type_ids = torch.tensor(input_type).to(device).reshape(-1, max_seq_length)
    
    with torch.set_grad_enabled(False):
        outputs = model(input_ids, token_type_ids, attention_mask)
        outputs = F.softmax(outputs,dim=1)
        df2.loc[index, 'sentiment'] = labels[torch.argmax(outputs).item()]

  0%|          | 0/230314 [00:00<?, ?it/s]

/home/ubuntu/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [42]:
df2.head(100)

,id,user,fullname,url,timestamp,replies,likes,retweets,text,language,probability,sentiment
11,1130878355375501313,giving_airdrops,Daily_Airdrops,NaN,2019-05-21 16:49:45+00:00,47.0,81.0,84.0,"BTC IS STILL GOING STRONG!!\n\nThus, we are gi...",__label__en,0.589066,positive
14,1132922663113023489,CCNMarkets,CCN.com,NaN,2019-05-27 08:13:06+00:00,5.0,167.0,68.0,"Bitcoin Price Hits $8,939 in New 2019 High: Wh...",__label__en,0.627685,neutral
19,1132927455985979393,giving_airdrops,Daily_Airdrops,NaN,2019-05-27 08:32:08+00:00,14.0,40.0,39.0,"BTC IS GOING CRAZYYY!\n\nThus, we are giving a...",__label__en,0.623620,neutral
20,1132752137581203456,BerlyChain,Berly Chain,NaN,2019-05-26 20:55:29+00:00,1.0,28.0,4.0,$HOT $HOT $HOT 🍀\n\n🚀🚀🚀🚀🚀\n\n🔥🔥🔥🔥🔥 https://t.c...,__label__en,0.832325,neutral
21,1110302988,halfin,halfin,NaN,2009-01-11 03:33:52+00:00,790.0,14470.0,5542.0,Running bitcoin,__label__en,0.276370,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...
690,1126565574891528192,CalvinAyre,Calvin Ayre,NaN,2019-05-10 11:22:26+00:00,24.0,81.0,12.0,hahaha....here is my view of the industry for ...,__label__en,0.844503,neutral
693,1132918100481986560,GoldenFarm12,Golden Farm Giveaways & Make Money Game,NaN,2019-05-27 07:54:58+00:00,48.0,54.0,59.0,💸[BTC] Bitcoin Giveaway 1.5 Bitcoin\nName: Bit...,__label__en,0.538141,neutral
694,1132740804374663168,CCNMarkets,CCN.com,NaN,2019-05-26 20:10:27+00:00,26.0,761.0,290.0,"Bitcoin Price Slays $8750, Hits 12-Month High ...",__label__en,0.539955,neutral
714,1132676199296196609,blockbarrister,Blockchain Barrister⚡️🇬🇧,NaN,2019-05-26 15:53:44+00:00,3.0,65.0,48.0,People like Vayes completely ignore the facts....,__label__en,0.872216,neutral


In [43]:
df2.to_csv('../tweets_20likes_eng_processed.csv', index=False)

In [44]:
df2['neutral'] = (df2['sentiment'] == 'neutral').astype(int)
df2['positive'] = (df2['sentiment'] == 'positive').astype(int)
df2['negative'] = (df2['sentiment'] == 'negative').astype(int)
df2['sentiment'].value_counts()

<ipython-input-44-98040d6a5bf4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['neutral'] = (df2['sentiment'] == 'neutral').astype(int)
<ipython-input-44-98040d6a5bf4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['positive'] = (df2['sentiment'] == 'positive').astype(int)
<ipython-input-44-98040d6a5bf4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

neutral     200760
positive     23811
negative      5743
Name: sentiment, dtype: int64

In [45]:
df2['date'] = pd.to_datetime(df2['timestamp']).dt.date
final = df2[['date', 'neutral', 'positive', 'negative']].groupby('date').sum().reset_index()
final

<ipython-input-45-b2f77548a316>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['date'] = pd.to_datetime(df2['timestamp']).dt.date


,date,neutral,positive,negative
0,2009-01-11,1,0,0
1,2009-01-21,1,0,0
2,2009-01-27,1,0,0
3,2009-01-29,1,0,0
4,2009-02-18,1,0,0
...,...,...,...,...
1547,2019-11-19,679,66,19
1548,2019-11-20,669,86,17
1549,2019-11-21,775,86,26
1550,2019-11-22,817,80,41


In [46]:
final.to_csv('../sentiment_count_20likes_eng.csv', index=False)